In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import glob
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm.notebook import tqdm

# Parsing

### Levels

In [4]:
geom_file = gpd.read_file(
    '/home/anton/dima_experiments/my_dissertation/meteo_grids_parser/data/russia_ws.gpkg')
geom_file = geom_file.set_index('gauge_id')

In [47]:
heigh_bs = pd.read_csv('./data/levels/height_id.csv')
heigh_bs = heigh_bs.rename(columns={'Unnamed: 0': 'gauge_id'})
heigh_bs['gauge_id'] = list(map(str, heigh_bs['gauge_id']))
heigh_bs = heigh_bs.set_index('gauge_id')
heigh_bs = heigh_bs.drop_duplicates()
heigh_bs


,name,height
gauge_id,,
75389,ПРОТВА - P.ПPOTBA - C.CПAC-ЗAГOPЬE,119.04
75394,ОСЕТР - P.OCETP - Д.MAPKИHO,107.00
75395,МОСКВА - P.MOCKBA - Д.БAPCУKИ,183.00
75402,МОСКВА - P.MOCKBA - Г.ЗBEHИГOPOД,132.89
75403,МОСКВА - P.MOCKBA - C.ПETPOВО-ДAЛЬHEE,127.50
...,...,...
49048,"Войница (Каарто-йоки, Терва-йоки) - P.BOЙHИЦA ...",101.17
49052,"Ухта (Ухтуа-йоки, Ухтуска) - P.УXTA - ПГT KAЛE...",114.32
49053,Р.НОРВИ-ЙОКИ - P.HOPBИ-ЙOKИ - ПГT KAЛEBAЛA,104.00


In [34]:
level_storage = './data/levels/final'
discharge_storage = './data/discharge/final'

In [36]:
lvl_file = pd.read_csv(f'{level_storage}/{gauge}.csv',
                       index_col='date')
lvl_file.index = pd.to_datetime(lvl_file.index)
lvl_bs = heigh_bs.loc[gauge, 'height']

lvl_file['lvl, mbs'] = lvl_bs + lvl_file['level']* 1e-2  # type: ignore
lvl_file = lvl_file.rename(columns={'level': 'lvl, sm'})
lvl_file

,"lvl, sm","lvl, mbs"
date,,
2008-01-01,53.0,119.57
2008-01-02,53.0,119.57
2008-01-03,54.0,119.58
2008-01-04,57.0,119.61
2008-01-05,57.0,119.61
...,...,...
2020-12-27,52.0,119.56
2020-12-28,55.0,119.59
2020-12-29,57.0,119.61


In [27]:
dis_file = pd.read_csv(f'{discharge_storage}/{gauge}.csv',
                    index_col='date')
dis_file.index = pd.to_datetime(dis_file.index)
ws_area = geom_file.loc[gauge, 'new_area']
dis_file['q, mm/day'] = (dis_file['discharge'] * 86400)/(ws_area * 1e3)
dis_file = dis_file.rename(columns={'discharge': 'q, cms/s'})
dis_file

,"q, cms/s","q, mm/day"
date,,
2008-01-01,11.3,0.269164
2008-01-02,11.3,0.269164
2008-01-03,11.3,0.269164
2008-01-04,11.3,0.269164
2008-01-05,11.1,0.264400
...,...,...
2020-12-27,NaN,NaN
2020-12-28,NaN,NaN
2020-12-29,NaN,NaN


In [31]:
res_file = pd.concat([dis_file, lvl_file], axis=1)
res_file = res_file[['lvl, sm', 'q, cms/s', 'lvl, mbs', 'q, mm/day']]
res_file

,"lvl, sm","q, cms/s","lvl, mbs","q, mm/day"
date,,,,
2008-01-01,53.0,11.3,119.57,0.269164
2008-01-02,53.0,11.3,119.57,0.269164
2008-01-03,54.0,11.3,119.58,0.269164
2008-01-04,57.0,11.3,119.61,0.269164
2008-01-05,57.0,11.1,119.61,0.264400
...,...,...,...,...
2020-12-27,52.0,NaN,119.56,NaN
2020-12-28,55.0,NaN,119.59,NaN
2020-12-29,57.0,NaN,119.61,NaN


In [28]:
pd.read_csv(f'{level_storage}/75389.csv',
            index_col='date')

,level
date,
2008-01-01,53.0
2008-01-02,53.0
2008-01-03,54.0
2008-01-04,57.0
2008-01-05,57.0
...,...
2020-12-27,52.0
2020-12-28,55.0
2020-12-29,57.0


In [42]:
gauge

'1001'

In [48]:
m_bs = heigh_bs.loc[gauge, 'height']
lvl_file = pd.read_csv(f'{level_storage}/{gauge}.csv',
                        index_col='date')
lvl_file.index = pd.to_datetime(lvl_file.index)
lvl_bs = heigh_bs.loc[gauge, 'height']
lvl_file['lvl, mbs'] = lvl_bs + \
    lvl_file['level'] * 1e-2  # type: ignore
lvl_file = lvl_file.rename(columns={'level': 'lvl, sm'})

In [49]:
lvl_bs

478.24

In [52]:
for gauge in heigh_bs.index:
    try:
        m_bs = heigh_bs.loc[gauge, 'height']
        lvl_file = pd.read_csv(f'{level_storage}/{gauge}.csv',
                               index_col='date')
        lvl_file.index = pd.to_datetime(lvl_file.index)
        lvl_bs = heigh_bs.loc[gauge, 'height']
        lvl_file['lvl, mbs'] = lvl_bs + \
            lvl_file['level'] * 1e-2  # type: ignore
        lvl_file = lvl_file.rename(columns={'level': 'lvl, sm'})
        lvl_file.to_csv(
            f'/home/anton/dima_experiments/geo_data/great_db/h/{gauge}.csv')
    except FileNotFoundError:
        lvl_file = pd.DataFrame()
        lvl_file.index = pd.date_range(start='01/01/2008',
                                       end='12/31/2020')
        lvl_file['lvl, sm'] = np.NaN
        lvl_file['lvl, mbs'] = np.NaN

    try:
        dis_file = pd.read_csv(f'{discharge_storage}/{gauge}.csv',
                               index_col='date')
        dis_file.index = pd.to_datetime(dis_file.index)
        try:
            ws_area = geom_file.loc[gauge, 'new_area']
        except KeyError:
            ws_area = np.NaN
        dis_file['q, mm/day'] = (dis_file['discharge'] * 86400)/(ws_area * 1e3)
        dis_file = dis_file.rename(columns={'discharge': 'q, cms/s'})
        dis_file.to_csv(
            f'/home/anton/dima_experiments/geo_data/great_db/q/{gauge}.csv')
    except FileNotFoundError:
        dis_file = pd.DataFrame()
        dis_file.index = pd.date_range(start='01/01/2008',
                                       end='12/31/2020')
        dis_file['q, cms/s'] = np.NaN
        dis_file['q, mm/day'] = np.NaN

    res_file = pd.concat([dis_file, lvl_file], axis=1)
    res_file = res_file[['lvl, sm', 'q, cms/s', 'lvl, mbs', 'q, mm/day']]
    res_file.to_csv(
        f'/home/anton/dima_experiments/geo_data/great_db/q_h/{gauge}.csv')
